# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

In [14]:
EPHYS_SAMPLE_RATE = 20000

Explanation of each input and where it comes from.

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_alone_comp"

In [16]:
# TRIAL_LABELS_DF = pd.read_excel("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/data/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")
TRIAL_LABELS_DF = pd.read_excel("../../data/sleap_id_corrected/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")

In [17]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_04_spectral_and_sleap.pkl".format(OUTPUT_PREFIX))

In [18]:
FULL_LFP_TRACES_PKL = "{}_10_per_trial_spectral_bans_sleap.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [19]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [20]:
LFP_SPECTRAL_DF.columns

Index(['video_name', 'current_subject', 'frame_index', 'tone_start_frame',
       'tone_stop_frame', 'condition ', 'competition_closeness', 'notes',
       '10s_before_tone_frame', '10s_after_tone_frame',
       ...
       'vHPC_RMS_filtered_power_gamma', 'vHPC_RMS_filtered_power_theta',
       'vHPC_gamma_band', 'vHPC_gamma_phase', 'vHPC_lfp_trace',
       'vHPC_mPFC_granger_all_frequencies_all_windows', 'vHPC_theta_band',
       'vHPC_theta_phase', 'video_timestamps', 'cluster_timestamp'],
      dtype='object', length=196)

In [21]:
TRIAL_LABELS_DF.columns

Index(['Unnamed: 0', 'session_dir', 'tone_start_frame', 'reward_start',
       'reward_dispensed', 'tone_stop_frame', 'condition ',
       'competition_closeness', 'get_reward_frame', 'out_reward_frame',
       'notes', 'experiment', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'tone_start_timestamp',
       'tone_stop_timestamp', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps'],
      dtype='object')

## Merging the labels and LFP spectral

In [22]:
TRIAL_LABELS_DF.head()

,Unnamed: 0,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,...,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,0.0,80,100.0,74.0,NaN,NaN,NaN,NaN,...,[ 0 74],[ 0 74],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,0.0,75474.0,[ 0 75474],[ 0 75474]
1,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,1272.0,1352,1372.0,1471.0,NaN,NaN,NaN,NaN,...,[1282 1283] [1285 1297] [1326 1344] [1362 1509],[1326 1328] [1331 1347] [1380 1592],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,1275889.0,1475891.0,[1285888 1286688] [1287488 1300689] [1330286 1...,[1329489 1331487] [1334489 1350089] [1383490 1...
2,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,3268.0,3348,3368.0,3469.0,NaN,NaN,NaN,NaN,...,[3281 3324] [3324 3327] [3344 3347] [3347 3523],[3327 3704],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,3275911.0,3475914.0,[3287914 3330314] [3331114 3333712] [3351715 3...,[3333515 3712519]
3,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,4466.0,4546,4566.0,4667.0,NaN,NaN,NaN,NaN,...,[4441 4476] [4476 4489] [4508 4509] [4511 4534...,[4445 4473] [4494 4551] [4566 4830],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,4475927.0,4675931.0,[4450528 4485129] [4485929 4497529] [4516929 4...,[4454926 4483129] [4503531 4560730] [4575330 4...
4,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,5465.0,5545,5565.0,5664.0,NaN,NaN,NaN,NaN,...,[5483 5512] [5552 5698],[5455 5486] [5581 5794],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,5475942.0,5675942.0,[5493742 5523542] [5563742 5709147],[5466941 5496741] [5592343 5806546]


In [23]:
LFP_SPECTRAL_DF.head()

,video_name,current_subject,frame_index,tone_start_frame,tone_stop_frame,condition,competition_closeness,notes,10s_before_tone_frame,10s_after_tone_frame,...,vHPC_RMS_filtered_power_gamma,vHPC_RMS_filtered_power_theta,vHPC_gamma_band,vHPC_gamma_phase,vHPC_lfp_trace,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_theta_band,vHPC_theta_phase,video_timestamps,cluster_timestamp
0,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,"[34042, 34043, 34044, 34045, 34046, 34047, 340...","[34242, 34242, 34242, 34242, 34242, 34242, 342...","[34442, 34442, 34442, 34442, 34442, 34442, 344...","[4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, ...","[After trial, After trial, After trial, After ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[34642, 34642, 34642, 34642, 34642, 34642, 346...",...,"[nan, 0.0015774673, 0.0016509024, 0.001697123,...","[nan, 0.029861301, 0.033973657, 0.05462378, 0....","[-0.06929627, -0.11967317, -0.15855941, -0.183...","[2.0203476, 2.3698096, 2.632465, 2.9235487, -3...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
1,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.3,"[34042, 34043, 34044, 34045, 34046, 34047, 340...","[34242, 34242, 34242, 34242, 34242, 34242, 342...","[34442, 34442, 34442, 34442, 34442, 34442, 344...","[4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, ...","[After trial, After trial, After trial, After ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[34642, 34642, 34642, 34642, 34642, 34642, 346...",...,"[0.00068537367, 0.0010442059, 0.0012645082, na...","[0.02813119, 0.023265623, 0.024091369, nan, na...","[0.056090515, 0.050495435, 0.040407084, 0.0263...","[-0.41799748, 0.42563042, 0.6898888, 1.115356,...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[[0.10801135, 0.9345647, 0.71951747, 0.1227541...","[-0.19497989, -0.21380065, -0.23236576, -0.250...","[2.0032697, 2.1665337, 2.1993783, 2.2978027, 2...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
2,20240322_120625_alone_comp_subj_3-3_and_3-4.1,3.3,"[34069, 34070, 34071, 34072, 34073, 34074, 340...","[34269, 34269, 34269, 34269, 34269, 34269, 342...","[34468, 34468, 34468, 34468, 34468, 34468, 344...","[3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, ...","[Subj 1 then Subj 2, Subj 1 then Subj 2, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34069, 34069, 34069, 34069, 34069, 34069, 340...","[34668, 34668, 34668, 34668, 34668, 34668, 346...",...,"[0.0014011, 0.0016158995, 0.0015800084, nan, n...","[0.013402096, 0.01478837, 0.009064879, nan, na...","[0.09273324, 0.093012325, 0.088457294, 0.07888...","[-0.5981963, 0.07866546, 0.2493537, 0.62838745...","[-24.765, 14.82, 24.375, -24.765, -36.855, -13...","[[0.014051933, 0.6209912, 0.20073363, 0.871971...","[0.069354706, 0.071877405, 0.07377358, 0.07502...","[-0.8432274, -0.42268786, -0.36932397, -0.1285...","[1384, 1983, 2770, 4156, 5541, 5541, 6927, 831...","[34103476, 34104862, 34106248, 34106248, 34107..."
3,20240322_120625_alone_comp_subj_3-3_and_3-4.1,3.4,"[34069, 34070, 34071, 34072, 34073, 34074, 340...","[34269, 34269, 34269, 34269, 34269, 34269, 342...","[34468, 34468, 34468, 34468, 34468, 34468, 344...","[3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, ...","[Subj 1 then Subj 2, Subj 1 then Subj 2, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34069, 34069, 34069, 34069, 34069, 34069, 340...","[34668, 34668, 34668, 34668, 34668, 34668, 346...",...,"[nan, nan, 0.0016887838, 0.002533161, 0.002136...","[nan, nan, 0.01613984, 0.013336834, 0.01096460...","[-0.39455938, -0.3748218, -0.33224645, -0.2686...","[2.607048, -2.9

# Dropping redundant columns

In [24]:
dropping_columns = list(set(TRIAL_LABELS_DF.columns).intersection(set(LFP_SPECTRAL_DF.columns)))

In [25]:
dropping_columns

['box_2_port_entry_timestamps',
 'tone_stop_frame',
 'session_dir',
 'box_1_port_entry_frames',
 'first_timestamp',
 'box_1_port_entry_timestamps',
 'last_timestamp',
 'box_2_port_entry_frames',
 'tone_start_frame',
 'all_subjects',
 'condition ',
 'competition_closeness',
 'experiment',
 'video_name',
 'notes']

In [26]:
dropping_columns = ['tone_start_frame', 'tone_stop_frame', 'condition ', 'notes', 'competition_closeness', '10s_before_tone_frame', '10s_after_tone_frame']

In [27]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.drop(columns=dropping_columns, errors="ignore")

In [28]:
dropping_columns = list(set(TRIAL_LABELS_DF.columns).intersection(set(LFP_SPECTRAL_DF.columns)))

In [29]:
LFP_SPECTRAL_DF["first_timestamp"].iloc[0]

2067718

In [30]:
dropping_columns

['box_2_port_entry_timestamps',
 'session_dir',
 'box_1_port_entry_frames',
 'first_timestamp',
 'box_1_port_entry_timestamps',
 'last_timestamp',
 'box_2_port_entry_frames',
 'all_subjects',
 'experiment',
 'video_name']

In [31]:
dropping_columns = ['experiment',
                    'box_1_port_entry_frames'
                    'box_1_port_entry_timestamps'
                    'box_2_port_entry_frames'
                    'box_2_port_entry_timestamps',
                    'session_dir',
                    'all_subjects',
                    'first_timestamp',
                    'last_timestamp']


In [32]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop(columns=dropping_columns, errors="ignore")

## Merging the LFP and trial dataframe

In [33]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [34]:
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str)


In [35]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["video_name"])

In [36]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition "])

In [37]:
TRIAL_LABELS_DF["video_name"] = TRIAL_LABELS_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [38]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1'], dtype=object)

In [39]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1'], dtype=object)

In [40]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["video_name"], how="inner", suffixes=('', '_remove'))

In [41]:
TRIALS_AND_SPECTRAL_DF[[col for col in TRIALS_AND_SPECTRAL_DF if "_remove" in col]].head()

,box_1_port_entry_frames_remove,box_1_port_entry_timestamps_remove,box_2_port_entry_frames_remove,box_2_port_entry_timestamps_remove
0,"[[0, 79], [84, 92], [92, 132], [145, 175], [25...","[[0, 79744], [85741, 92941], [93544, 134344], ...","[[0, 79], [79, 186], [188, 191], [191, 268], [...","[[0, 79744], [80141, 188142], [188942, 191545]..."
1,"[[0, 79], [84, 92], [92, 132], [145, 175], [25...","[[0, 79744], [85741, 92941], [93544, 134344], ...","[[0, 79], [79, 186], [188, 191], [191, 268], [...","[[0, 79744], [80141, 188142], [188942, 191545]..."
2,"[[0, 79], [84, 92], [92, 132], [145, 175], [25...","[[0, 79744], [85741, 92941], [93544, 134344], ...","[[0, 79], [79, 186], [188, 191], [191, 268], [...","[[0, 79744], [80141, 188142], [188942, 191545]..."
3,"[[0, 79], [84, 92], [92, 132], [145, 175], [25...","[[0, 79744], [85741, 92941], [93544, 134344], ...","[[0, 79], [79, 186], [188, 191], [191, 268], [...","[[0, 79744], [80141, 188142], [188942, 191545]..."
4,"[[0, 79], [84, 92], [92, 132], [145, 175], [25...","[[0, 79744], [85741, 92941], [93544, 134344], ...","[[0, 79], [79, 186], [188, 191], [191, 268], [...","[[0, 79744], [80141, 188142], [188942, 191545]..."


In [42]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "remove" in col], errors="ignore")

In [43]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF if "Unnamed" in col])

In [44]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,box_1_port_entry_frames,...,vHPC_RMS_filtered_power_gamma,vHPC_RMS_filtered_power_theta,vHPC_gamma_band,vHPC_gamma_phase,vHPC_lfp_trace,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_theta_band,vHPC_theta_phase,video_timestamps,cluster_timestamp
0,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,[34250 34265],...,"[nan, 0.0015774673, 0.0016509024, 0.001697123,...","[nan, 0.029861301, 0.033973657, 0.05462378, 0....","[-0.06929627, -0.11967317, -0.15855941, -0.183...","[2.0203476, 2.3698096, 2.632465, 2.9235487, -3...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
1,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,[34250 34265],...,"[0.00068537367, 0.0010442059, 0.0012645082, na...","[0.02813119, 0.023265623, 0.024091369, nan, na...","[0.056090515, 0.050495435, 0.040407084, 0.0263...","[-0.41799748, 0.42563042, 0.6898888, 1.115356,...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[[0.10801135, 0.9345647, 0.71951747, 0.1227541...","[-0.19497989, -0.21380065, -0.23236576, -0.250...","[2.0032697, 2.1665337, 2.1993783, 2.2978027, 2...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
2,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,[35242 35249] [35249 35250] [35250 35271] [353...,...,"[nan, 0.0015774673, 0.0016509024, 0.001697123,...","[nan, 0.029861301, 0.033973657, 0.05462378, 0....","[-0.06929627, -0.11967317, -0.15855941, -0.183...","[2.0203476, 2.3698096, 2.632465, 2.9235487, -3...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,[35242 35249] [35249 35250] [35250 35271] [353...,...,"[0.00068537367, 0.0010442059, 0.0012645082, na...","[0.02813119, 0.023265623, 0.024091369, nan, na...","[0.056090515, 0.050495435, 0.040407084, 0.0263...","[-0.41799748, 0.42563042, 0.6898888, 1.115356,...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[[0.10801135, 0.9345647, 0.71951747, 0.1227541...","[-0.19497989, -0.21380065, -0.23236576, -0.250...","[2.0032697, 2.1665337, 2.1993783, 2.2978027, 2...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."
4,36339.0,36419,36439.0,36538.0,4.3,Subj 2 blocking Subj 1,NaN,NaN,NaN,[36339 36354] [36361 36365] [36410 36473] [364...,...,"[nan, 0.0015774673, 0.0016509024, 0.001697123,...","[nan, 0.029861301, 0.033973657, 0.05462378, 0....","[-0.06929627, -0.11967317, -0.15855941, -0.183...","[2.0203476, 2.3698096, 2.632465, 2.9235487, -3...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084..."


In [45]:
TRIALS_AND_SPECTRAL_DF.tail()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,box_1_port_entry_frames,...,vHPC_RMS_filtered_power_gamma,vHPC_RMS_filtered_power_theta,vHPC_gamma_band,vHPC_gamma_phase,vHPC_lfp_trace,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_theta_band,vHPC_theta_phase,video_timestamps,cluster_timestamp
243,57880.0,57960,57980.0,58079.0,4.4,Subj 2 then Subj 1,NaN,NaN,NaN,[57883 57894] [57896 57900] [57901 57909] [579...,...,"[0.0026200346, nan, nan, 0.0027044346, 0.00204...","[0.015349463, nan, nan, 0.0061448426, 0.006152...","[0.08948469, -0.002254758, -0.09536301, -0.182...","[0.88585144, 1.5805309, 1.9701416, 2.2596772, ...","[5.85, 3.3149998, -7.7999997, -18.914999, -17....","[[0.22426029, 0.031661306, 0.23187897, 0.50685...","[-0.17116006, -0.19133626, -0.21119694, -0.230...","[1.9537605, 2.088994, 2.1246827, 2.2100766, 2....","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186..."
244,60077.0,60157,60177.0,60276.0,4.4,Subj 2 blocking Subj 1,NaN,NaN,4.2 just jumping around on the right side of port,[60070 60098] [60099 60101] [60103 60105] [601...,...,"[0.0015146171, 0.0020185008, 0.0026215008, 0.0...","[0.042833474, 0.0325564, 0.014915451, 0.023749...","[0.124277644, 0.10781112, 0.0808986, 0.0448804...","[-0.32664496, 0.55800784, 0.84448737, 1.252265...","[8.19, 31.005, 57.719997, 70.005, 64.935, 24.1...","[[3.099082, 0.46414718, 0.33549076, 0.6002563,...","[-0.2993394, -0.27316827, -0.24553603, -0.2164...","[3.108997, -2.4551952, -2.344967, -2.1353333, ...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186..."
245,60077.0,60157,60177.0,60276.0,4.4,Subj 2 blocking Subj 1,NaN,NaN,4.2 just jumping around on the right side of port,[60070 60098] [60099 60101] [60103 60105] [601...,...,"[0.0026200346, nan, nan, 0.0027044346, 0.00204...","[0.015349463, nan, nan, 0.0061448426, 0.006152...","[0.08948469, -0.002254758, -0.09536301, -0.182...","[0.88585144, 1.5805309, 1.9701416, 2.2596772, ...","[5.85, 3.3149998, -7.7999997, -18.914999, -17....","[[0.22426029, 0.031661306, 0.23187897, 0.50685...","[-0.17116006, -0.19133626, -0.21119694, -0.230...","[1.9537605, 2.088994, 2.1246827, 2.2100766, 2....","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186..."
246,61174.0,61254,61274.0,61375.0,tie,Close Call,NaN,NaN,NaN,[61188 61211] [61211 61214] [61214 61220] [612...,...,"[0.0015146171, 0.0020185008, 0.0026215008, 0.0...","[0.042833474, 0.0325564, 0.014915451, 0.023749...","[0.124277644, 0.10781112, 0.0808986, 0.0448804...","[-0.32664496, 0.55800784, 0.84448737, 1.252265...","[8.19, 31.005, 57.719997, 70.005, 64.935, 24.1...","[[3.099082, 0.46414718, 0.33549076, 0.6002563,...","[-0.2993394, -0.27316827, -0.24553603, -0.2164...","[3.108997, -2.4551952, -2.344967, -2.1353333, ...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186..."
247,61174.0,61254,61274.0,61375.0,tie,Close Call,NaN,NaN,NaN,[61188 61211] [61211 61214] [61214 61220] [612...,...,"[0.0026200346, nan, nan, 0.0027044346, 0.00204...","[0.015349463, nan, nan, 0.0061448426, 0.006152...","[0.08948469, -0.002254758, -0.09536301, -0.182...","[0.88585144, 1.5805309, 1.9701416, 2.2596772, ...","[5.85, 3.3149998, -7.7999997, -18.914999, -17....","[[0.22426029, 0.031661306, 0.23187897, 0.50685...","[-0.17116006, -0.19133626, -0.21119694, -0.230...","[1.9537605, 2.088994, 2.1246827, 2.2100766, 2....","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186..."


In [46]:
timestamps_col = [col for col in TRIALS_AND_SPECTRAL_DF if "timestamp" in col and "timestamps" not in col and "cluster" not in col]

In [47]:
for col in timestamps_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF[col].astype(int)

In [48]:
TRIALS_AND_SPECTRAL_DF[timestamps_col].head()

,tone_start_timestamp,tone_stop_timestamp,first_timestamp,last_timestamp
0,34280576,34480576,2067718,66452444
1,34280576,34480576,2067718,66452444
2,35280586,35480585,2067718,66452444
3,35280586,35480585,2067718,66452444
4,36380600,36580602,2067718,66452444


# Calculating the relevant timestamps

In [49]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 10 * EPHYS_SAMPLE_RATE

In [50]:
TRIALS_AND_SPECTRAL_DF["post_trial_end_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"] + 10 * EPHYS_SAMPLE_RATE

In [51]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    34280576
1    34280576
2    35280586
3    35280586
4    36380600
Name: tone_start_timestamp, dtype: int64

In [52]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    34280576
1    34280576
2    35280586
3    35280586
4    36380600
Name: tone_start_timestamp, dtype: int64

In [53]:
TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"].head()

0    34480576
1    34480576
2    35480585
3    35480585
4    36580602
Name: tone_stop_timestamp, dtype: int64

In [54]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,box_1_port_entry_frames,...,vHPC_gamma_band,vHPC_gamma_phase,vHPC_lfp_trace,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_theta_band,vHPC_theta_phase,video_timestamps,cluster_timestamp,baseline_start_timestamp,post_trial_end_timestamp
0,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,[34250 34265],...,"[-0.06929627, -0.11967317, -0.15855941, -0.183...","[2.0203476, 2.3698096, 2.632465, 2.9235487, -3...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...",34080576,34680576
1,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,[34250 34265],...,"[0.056090515, 0.050495435, 0.040407084, 0.0263...","[-0.41799748, 0.42563042, 0.6898888, 1.115356,...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[[0.10801135, 0.9345647, 0.71951747, 0.1227541...","[-0.19497989, -0.21380065, -0.23236576, -0.250...","[2.0032697, 2.1665337, 2.1993783, 2.2978027, 2...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...",34080576,34680576
2,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,[35242 35249] [35249 35250] [35250 35271] [353...,...,"[-0.06929627, -0.11967317, -0.15855941, -0.183...","[2.0203476, 2.3698096, 2.632465, 2.9235487, -3...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...",35080586,35680585
3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,[35242 35249] [35249 35250] [35250 35271] [353...,...,"[0.056090515, 0.050495435, 0.040407084, 0.0263...","[-0.41799748, 0.42563042, 0.6898888, 1.115356,...","[-0.78, -3.8999999, -51.675, -83.85, -19.89, 6...","[[0.10801135, 0.9345647, 0.71951747, 0.1227541...","[-0.19497989, -0.21380065, -0.23236576, -0.250...","[2.0032697, 2.1665337, 2.1993783, 2.2978027, 2...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...",35080586,35680585
4,36339.0,36419,36439.0,36538.0,4.3,Subj 2 blocking Subj 1,NaN,NaN,NaN,[36339 36354] [36361 36365] [36410 36473] [364...,...,"[-0.06929627, -0.11967317, -0.15855941, -0.183...","[2.0203476, 2.3698096, 2.632465, 2.9235487, -3...","[-8.384999, -70.005, -99.645, -99.45, -157.56,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[0.6103468, 0.5730442, 0.5342873, 0.4941693, 0...","[-0.41403934, 0.30454272, 0.38569367, 0.659321...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...",36180600,36780602


# Filtering out the SLEAP posed for during trials

In [55]:
TRIALS_AND_SPECTRAL_DF["video_frame"] = TRIALS_AND_SPECTRAL_DF["video_timestamps"].apply(lambda x: np.array(list(range(len(x)))) + 1)

In [56]:
TRIALS_AND_SPECTRAL_DF["video_frame"].head().apply(lambda x: x.shape)

0    (64255,)
1    (64255,)
2    (64255,)
3    (64255,)
4    (64255,)
Name: video_frame, dtype: object

In [57]:
['subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'manual_cluster_id',
 'standard_embedding_x',
 'standard_embedding_y',
 'kmeans_cluster',
 'subject_locations',
 'agent_locations',
 'subject_thorax',
 'subject_nose',
 'subject_tail_base',
 'agent_thorax',
 'agent_nose',
 'agent_tail_base']

['subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'manual_cluster_id',
 'standard_embedding_x',
 'standard_embedding_y',
 'kmeans_cluster',
 'subject_locations',
 'agent_locations',
 'subject_thorax',
 'subject_nose',
 'subject_tail_base',
 'agent_thorax',
 'agent_nose',
 'agent_tail_base']

In [58]:
# sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "locations" in col or "velocity" in col or "to_reward_port" in col or "video_frame" in col]

In [59]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if ("thorax" in col or "nose" in col or "reward_port" in col or "standard_embedding" in col or "cluster" in col or "frame_index" in col or "locations" in col or "tailbase" in col) and "timestamp" not in col]


In [60]:
sleap_columns

['frame_index',
 'subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'manual_cluster_id',
 'standard_embedding_x',
 'standard_embedding_y',
 'kmeans_cluster',
 'subject_locations',
 'agent_locations',
 'subject_thorax',
 'subject_nose',
 'agent_thorax',
 'agent_nose']

In [61]:
for col in sleap_columns:
    print(col)
    print(TRIALS_AND_SPECTRAL_DF[col].apply(lambda x: x.shape).iloc[0])

frame_index
(12593,)
subject_thorax_to_agent_thorax
(12593,)
nose_to_reward_port_sum
(12593,)
nose_to_reward_port_diff
(12593,)
thorax_velocity_sum
(12593,)
thorax_velocity_diff
(12593,)
to_reward_port_angle_sum
(12593,)
to_reward_port_angle_diff
(12593,)
subject_nose_to_reward_port
(12593,)
subject_thorax_velocity
(12593,)
subject_to_reward_port_angle
(12593,)
agent_nose_to_reward_port
(12593,)
agent_thorax_velocity
(12593,)
agent_to_reward_port_angle
(12593,)
closebool_subject_nose_to_reward_port
(12593,)
closebool_agent_nose_to_reward_port
(12593,)
movingbool_subject_thorax_velocity
(12593,)
movingbool_agent_thorax_velocity
(12593,)
manual_cluster_id
(12593,)
standard_embedding_x
(12593,)
standard_embedding_y
(12593,)
kmeans_cluster
(12593,)
subject_locations
(12593, 6, 2)
agent_locations
(12593, 6, 2)
subject_thorax
(12593, 2)
subject_nose
(12593, 2)
agent_thorax
(12593, 2)
agent_nose
(12593, 2)


In [62]:
for col in sorted(sleap_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[0], axis=1)

trial_and_post_agent_locations
trial_and_post_agent_nose
trial_and_post_agent_nose_to_reward_port
trial_and_post_agent_thorax
trial_and_post_agent_thorax_velocity
trial_and_post_agent_to_reward_port_angle
trial_and_post_closebool_agent_nose_to_reward_port
trial_and_post_closebool_subject_nose_to_reward_port
trial_and_post_frame_index
trial_and_post_kmeans_cluster
trial_and_post_manual_cluster_id
trial_and_post_movingbool_agent_thorax_velocity
trial_and_post_movingbool_subject_thorax_velocity
trial_and_post_nose_to_reward_port_diff
trial_and_post_nose_to_reward_port_sum
trial_and_post_standard_embedding_x
trial_and_post_standard_embedding_y
trial_and_post_subject_locations
trial_and_post_subject_nose
trial_and_post_subject_nose_to_reward_port
trial_and_post_subject_thorax
trial_and_post_subject_thorax_to_agent_thorax
trial_and_post_subject_thorax_velocity
trial_and_post_subject_to_reward_port_angle
trial_and_post_thorax_velocity_diff
trial_and_post_thorax_velocity_sum
trial_and_post_to_

In [63]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,box_1_port_entry_frames,...,trial_and_post_subject_nose_to_reward_port,trial_and_post_subject_thorax,trial_and_post_subject_thorax_to_agent_thorax,trial_and_post_subject_thorax_velocity,trial_and_post_subject_to_reward_port_angle,trial_and_post_thorax_velocity_diff,trial_and_post_thorax_velocity_sum,trial_and_post_to_reward_port_angle_diff,trial_and_post_to_reward_port_angle_sum,trial_and_post_video_timestamps
0,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,[34250 34265],...,"[2.4861953, 2.2716153, 2.0270197, 1.8338053, 1...","[[71.95774040324862, 32.322139769886114], [71....","[13.249745, 12.420274, 11.498921, 10.627059, 9...","[4.569111, 5.1593533, 5.675921, 6.0645876, 6.3...","[2.9412072, 2.8742726, 2.7693202, 2.6417112, 2...","[5.8270407, 7.315352, 9.172596, 11.392724, 13....","[14.965262, 17.634058, 20.524439, 23.5219, 26....","[1.9850475, 2.0960886, 2.1170332, 2.0739496, 1...","[3.9031072, 3.6548777, 3.3964436, 3.159185, 2....","[34280867, 34282253, 34282253, 34283639, 34285..."
1,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,[34250 34265],...,"[6.6279693, 7.402137, 8.201156, 9.028512, 9.92...","[[57.85368672899245, 29.88676433471742], [58.3...","[13.249745, 12.420274, 11.498921, 10.627059, 9...","[10.396152, 12.474705, 14.848517, 17.457312, 2...","[0.96189994, 0.7806051, 0.62712336, 0.5174737,...","[5.8270407, 7.315352, 9.172596, 11.392724, 13....","[14.965262, 17.634058, 20.524439, 23.5219, 26....","[1.9850475, 2.0960886, 2.1170332, 2.0739496, 1...","[3.9031072, 3.6548777, 3.3964436, 3.159185, 2....","[34280867, 34282253, 34282253, 34283639, 34285..."
2,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,[35242 35249] [35249 35250] [35250 35271] [353...,...,"[1.1089377, 1.1452469, 1.1488823, 1.1699165, 1...","[[59.889793975715484, 31.176297534826535], [59...","[27.067476, 26.779356, 26.523151, 26.297247, 2...","[0.40238845, 0.4286343, 0.4387325, 0.44147334,...","[0.27336645, 0.31294948, 0.35071996, 0.3853223...","[4.3826394, 3.919239, 3.5807028, 3.4981563, 3....","[5.187416, 4.7765074, 4.4581676, 4.3811026, 4....","[0.19098243, 0.19574554, 0.22000165, 0.2532908...","[0.31987455, 0.38714713, 0.47364113, 0.5777477...","[35281465, 35282851, 35284237, 35284237, 35285..."
3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,[35242 35249] [35249 35250] [35250 35271] [353...,...,"[36.418747, 36.07958, 35.682335, 35.189987, 34...","[[52.097475233035375, 4.403293847774247], [52....","[27.067476, 26.779356, 26.523151, 26.297247, 2...","[4.785028, 4.347873, 4.0194354, 3.9396296, 4.2...","[0.04650808, 0.07419765, 0.12292117, 0.1924254...","[4.3826394, 3.919239, 3.5807028, 3.4981563, 3....","[5.187416, 4.7765074, 4.4581676, 4.3811026, 4....","[0.19098243, 0.19574554, 0.22000165, 0.2532908...","[0.31987455, 0.38714713, 0.47364113, 0.5777477...","[35281465, 35282851, 35284237, 35284237, 35285..."
4,36339.0,36419,36439.0,36538.0,4.3,Subj 2 blocking Subj 1,NaN,NaN,NaN,[36339 36354] [36361 36365] [36410 36473] [364...,...,"[1.1306264, 0.97184813, 0.85596865, 0.58294195...","[[59.88265859041628, 30.855202660299007], [60....","[9.941473, 9.860621, 9.8498745, 9.888419, 9.96...","[1.1130899, 1.0205495, 0.9626311, 0.9428376, 0...","[0.83984494, 0.8005584, 0.71527505, 0.64785486...","[4.92766, 5.32972, 5.5500345, 5.5871577, 5.489...","[6.954897, 7.253446, 7.432253, 7.5142064, 7.54...","[0.54289323, 0.6733394, 0.8578373, 1.0346372, ...","[1.9546683, 2.028851, 2.0825138, 2.171481, 2.2...","[36381846, 36383232, 36383232, 36384617, 36386..."


In [64]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_frame_index"].iloc[0].shape

(400,)

In [65]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["cluster_timestamp"], errors="ignore")

In [66]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition 
competition_closeness
get_reward_frame
out_reward_frame
notes
box_1_port_entry_frames
box_2_port_entry_frames
video_name
tone_start_timestamp
tone_stop_timestamp
box_1_port_entry_timestamps
box_2_port_entry_timestamps
current_subject
subject_tail_base
agent_tail_base
session_dir
experiment
sleap_name
video_id
agent
BLA-to-LH_granger_gamma
BLA-to-LH_granger_theta
BLA-to-MD_granger_gamma
BLA-to-MD_granger_theta
BLA-to-mPFC_granger_gamma
BLA-to-mPFC_granger_theta
BLA-to-vHPC_granger_gamma
BLA-to-vHPC_granger_theta
BLA_LH_coherence_all_frequencies_all_windows
BLA_LH_coherence_gamma
BLA_LH_coherence_theta
BLA_LH_granger_all_frequencies_all_windows
BLA_MD_coherence_all_frequencies_all_windows
BLA_MD_coherence_gamma
BLA_MD_coherence_theta
BLA_MD_granger_all_frequencies_all_windows
BLA_RMS_filtered_power_all_frequencies_all_windows
BLA_RMS_filtered_power_gamma
BLA_RMS_filtered_power_theta
BLA_gamma_band
BLA_gamma_phase
B

- Filtering coherence

In [67]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [68]:
coherence_columns

['BLA_LH_coherence_all_frequencies_all_windows',
 'BLA_LH_coherence_gamma',
 'BLA_LH_coherence_theta',
 'BLA_MD_coherence_all_frequencies_all_windows',
 'BLA_MD_coherence_gamma',
 'BLA_MD_coherence_theta',
 'BLA_mPFC_coherence_all_frequencies_all_windows',
 'BLA_mPFC_coherence_gamma',
 'BLA_mPFC_coherence_theta',
 'BLA_vHPC_coherence_all_frequencies_all_windows',
 'BLA_vHPC_coherence_gamma',
 'BLA_vHPC_coherence_theta',
 'LH_MD_coherence_all_frequencies_all_windows',
 'LH_MD_coherence_gamma',
 'LH_MD_coherence_theta',
 'LH_mPFC_coherence_all_frequencies_all_windows',
 'LH_mPFC_coherence_gamma',
 'LH_mPFC_coherence_theta',
 'LH_vHPC_coherence_all_frequencies_all_windows',
 'LH_vHPC_coherence_gamma',
 'LH_vHPC_coherence_theta',
 'MD_mPFC_coherence_all_frequencies_all_windows',
 'MD_mPFC_coherence_gamma',
 'MD_mPFC_coherence_theta',
 'MD_vHPC_coherence_all_frequencies_all_windows',
 'MD_vHPC_coherence_gamma',
 'MD_vHPC_coherence_theta',
 'mPFC_vHPC_coherence_all_frequencies_all_windows',


In [69]:
for col in sorted(coherence_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_coherence_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

trial_and_post_BLA_LH_coherence_all_frequencies_all_windows
trial_and_post_BLA_LH_coherence_gamma
trial_and_post_BLA_LH_coherence_theta
trial_and_post_BLA_MD_coherence_all_frequencies_all_windows
trial_and_post_BLA_MD_coherence_gamma
trial_and_post_BLA_MD_coherence_theta
trial_and_post_BLA_mPFC_coherence_all_frequencies_all_windows
trial_and_post_BLA_mPFC_coherence_gamma
trial_and_post_BLA_mPFC_coherence_theta
trial_and_post_BLA_vHPC_coherence_all_frequencies_all_windows
trial_and_post_BLA_vHPC_coherence_gamma
trial_and_post_BLA_vHPC_coherence_theta
trial_and_post_LH_MD_coherence_all_frequencies_all_windows
trial_and_post_LH_MD_coherence_gamma
trial_and_post_LH_MD_coherence_theta
trial_and_post_LH_mPFC_coherence_all_frequencies_all_windows
trial_and_post_LH_mPFC_coherence_gamma
trial_and_post_LH_mPFC_coherence_theta
trial_and_post_LH_vHPC_coherence_all_frequencies_all_windows
trial_and_post_LH_vHPC_coherence_gamma
trial_and_post_LH_vHPC_coherence_theta
trial_and_post_MD_mPFC_coherence_

In [70]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [71]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [72]:
granger_columns

['BLA-to-LH_granger_gamma',
 'BLA-to-LH_granger_theta',
 'BLA-to-MD_granger_gamma',
 'BLA-to-MD_granger_theta',
 'BLA-to-mPFC_granger_gamma',
 'BLA-to-mPFC_granger_theta',
 'BLA-to-vHPC_granger_gamma',
 'BLA-to-vHPC_granger_theta',
 'BLA_LH_granger_all_frequencies_all_windows',
 'BLA_MD_granger_all_frequencies_all_windows',
 'BLA_mPFC_granger_all_frequencies_all_windows',
 'BLA_vHPC_granger_all_frequencies_all_windows',
 'LH-to-BLA_granger_gamma',
 'LH-to-BLA_granger_theta',
 'LH-to-MD_granger_gamma',
 'LH-to-MD_granger_theta',
 'LH-to-mPFC_granger_gamma',
 'LH-to-mPFC_granger_theta',
 'LH-to-vHPC_granger_gamma',
 'LH-to-vHPC_granger_theta',
 'LH_BLA_granger_all_frequencies_all_windows',
 'LH_MD_granger_all_frequencies_all_windows',
 'LH_mPFC_granger_all_frequencies_all_windows',
 'LH_vHPC_granger_all_frequencies_all_windows',
 'MD-to-BLA_granger_gamma',
 'MD-to-BLA_granger_theta',
 'MD-to-LH_granger_gamma',
 'MD-to-LH_granger_theta',
 'MD-to-mPFC_granger_gamma',
 'MD-to-mPFC_granger_t

In [73]:
for col in sorted(granger_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_granger_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

trial_and_post_BLA-to-LH_granger_gamma
trial_and_post_BLA-to-LH_granger_theta
trial_and_post_BLA-to-MD_granger_gamma
trial_and_post_BLA-to-MD_granger_theta
trial_and_post_BLA-to-mPFC_granger_gamma
trial_and_post_BLA-to-mPFC_granger_theta
trial_and_post_BLA-to-vHPC_granger_gamma
trial_and_post_BLA-to-vHPC_granger_theta
trial_and_post_BLA_LH_granger_all_frequencies_all_windows
trial_and_post_BLA_MD_granger_all_frequencies_all_windows
trial_and_post_BLA_mPFC_granger_all_frequencies_all_windows
trial_and_post_BLA_vHPC_granger_all_frequencies_all_windows
trial_and_post_LH-to-BLA_granger_gamma
trial_and_post_LH-to-BLA_granger_theta
trial_and_post_LH-to-MD_granger_gamma
trial_and_post_LH-to-MD_granger_theta
trial_and_post_LH-to-mPFC_granger_gamma
trial_and_post_LH-to-mPFC_granger_theta
trial_and_post_LH-to-vHPC_granger_gamma
trial_and_post_LH-to-vHPC_granger_theta
trial_and_post_LH_BLA_granger_all_frequencies_all_windows
trial_and_post_LH_MD_granger_all_frequencies_all_windows
trial_and_post_

In [74]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

In [75]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

- Filtering power

In [76]:
TRIALS_AND_SPECTRAL_DF.head()

,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,notes,box_1_port_entry_frames,...,trial_and_post_vHPC-to-LH_granger_theta,trial_and_post_vHPC-to-MD_granger_gamma,trial_and_post_vHPC-to-MD_granger_theta,trial_and_post_vHPC-to-mPFC_granger_gamma,trial_and_post_vHPC-to-mPFC_granger_theta,trial_and_post_vHPC_BLA_granger_all_frequencies_all_windows,trial_and_post_vHPC_LH_granger_all_frequencies_all_windows,trial_and_post_vHPC_MD_granger_all_frequencies_all_windows,trial_and_post_vHPC_mPFC_granger_all_frequencies_all_windows,trial_and_post_granger_timestamps
0,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,[34250 34265],...,"[nan, nan, 0.20589761, 0.9640081, nan, nan, 0....","[nan, nan, 0.30926424, 0.25220188, nan, nan, 0...","[nan, nan, 0.095124066, 0.643016, nan, nan, 0....","[nan, nan, 0.1569771, 0.19070423, 0.4585937, 0...","[nan, nan, 0.2088117, 0.16601166, 0.3455993, 0...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[34290000, 34300000, 34310000, 34320000, 34330..."
1,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,NaN,[34250 34265],...,"[0.10217985, 0.12709841, 0.20064971, 0.0970779...","[0.519983, 0.4223254, 0.2499345, 0.30090326, 0...","[0.10570881, 0.14331408, 0.17701852, 0.1849999...","[0.21263, 0.37446445, 0.34829086, 0.36602214, ...","[0.07440844, 0.10832663, 0.1621511, 0.0767037,...","[[0.9082437, 1.702321, 0.26233336, 0.005933128...","[[0.5064685, 0.358471, 0.55923617, 0.30875865,...","[[0.96538275, 0.10910119, 0.004785885, 0.01848...","[[0.016020196, 0.00012993268, 0.2842621, 0.199...","[34290000, 34300000, 34310000, 34320000, 34330..."
2,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,[35242 35249] [35249 35250] [35250 35271] [353...,...,"[nan, nan, 0.62891924, 0.6018706, 0.265078, 0....","[nan, nan, 0.57126826, 0.3705952, 0.49803594, ...","[nan, nan, 0.54193485, 0.2436925, 0.26329193, ...","[nan, nan, 0.166316, 0.31046996, 0.21204294, 0...","[nan, nan, 0.24754146, 0.15036911, 0.061768956...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[35290000, 35300000, 35310000, 35320000, 35330..."
3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,NaN,[35242 35249] [35249 35250] [35250 35271] [353...,...,"[0.13519448, 0.04071226, 0.113442235, 0.200003...","[0.28915203, 0.28622752, 0.15704502, 0.1970611...","[0.26806355, 0.2262254, 0.33601293, 0.81927377...","[0.17953686, 0.28731292, 0.23438458, 0.2861041...","[0.25073707, 0.21066107, 0.3490196, 0.07607516...","[[0.5816017, 0.063610435, 0.14955683, 0.210158...","[[0.6568974, 1.0212861, 0.05766481, 0.1917388,...","[[0.77305835, 0.23048328, 0.24249722, 0.269778...","[[0.027987318, 0.061369784, 0.01772461, 0.1635...","[35290000, 35300000, 35310000, 35320000, 35330..."
4,36339.0,36419,36439.0,36538.0,4.3,Subj 2 blocking Subj 1,NaN,NaN,NaN,[36339 36354] [36361 36365] [36410 36473] [364...,...,"[0.20555179, 0.54496163, 0.5135671, 0.4818653,...","[0.12461248, 0.17561705, 0.27625504, 0.1882012...","[0.20821989, 0.45550716, 0.2562119, 0.3394489,...","[0.17908934, 0.23997788, 0.39097258, 0.4240108...","[0.16767275, 0.311181, 0.3730203, 0.33727127, ...","[[0.004335904, 0.004469719, 0.019590594, 0.001...","[[0.18915232, 0.47623792, 0.19593622, 0.045745...","[[0.5212869, 0.5885494, 0.13759334, 0.05542526...","[[1.0416099, 0.33503333, 0.0035690519, 0.07738...","[36390000, 36400000, 36410000, 36420000, 36430..."


In [77]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [78]:
power_columns

['BLA_RMS_filtered_power_all_frequencies_all_windows',
 'BLA_RMS_filtered_power_gamma',
 'BLA_RMS_filtered_power_theta',
 'LH_RMS_filtered_power_all_frequencies_all_windows',
 'LH_RMS_filtered_power_gamma',
 'LH_RMS_filtered_power_theta',
 'MD_RMS_filtered_power_all_frequencies_all_windows',
 'MD_RMS_filtered_power_gamma',
 'MD_RMS_filtered_power_theta',
 'mPFC_RMS_filtered_power_all_frequencies_all_windows',
 'mPFC_RMS_filtered_power_gamma',
 'mPFC_RMS_filtered_power_theta',
 'vHPC_RMS_filtered_power_all_frequencies_all_windows',
 'vHPC_RMS_filtered_power_gamma',
 'vHPC_RMS_filtered_power_theta']

In [79]:
for col in sorted(power_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_power_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)


trial_and_post_BLA_RMS_filtered_power_all_frequencies_all_windows
trial_and_post_BLA_RMS_filtered_power_gamma
trial_and_post_BLA_RMS_filtered_power_theta
trial_and_post_LH_RMS_filtered_power_all_frequencies_all_windows
trial_and_post_LH_RMS_filtered_power_gamma
trial_and_post_LH_RMS_filtered_power_theta
trial_and_post_MD_RMS_filtered_power_all_frequencies_all_windows
trial_and_post_MD_RMS_filtered_power_gamma
trial_and_post_MD_RMS_filtered_power_theta
trial_and_post_mPFC_RMS_filtered_power_all_frequencies_all_windows
trial_and_post_mPFC_RMS_filtered_power_gamma
trial_and_post_mPFC_RMS_filtered_power_theta
trial_and_post_vHPC_RMS_filtered_power_all_frequencies_all_windows
trial_and_post_vHPC_RMS_filtered_power_gamma
trial_and_post_vHPC_RMS_filtered_power_theta


In [80]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

In [81]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition 
competition_closeness
get_reward_frame
out_reward_frame
notes
box_1_port_entry_frames
box_2_port_entry_frames
video_name
tone_start_timestamp
tone_stop_timestamp
box_1_port_entry_timestamps
box_2_port_entry_timestamps
current_subject
subject_tail_base
agent_tail_base
session_dir
experiment
sleap_name
video_id
agent
BLA_gamma_band
BLA_gamma_phase
BLA_lfp_trace
BLA_theta_band
BLA_theta_phase
LH_gamma_band
LH_gamma_phase
LH_lfp_trace
LH_theta_band
LH_theta_phase
MD_gamma_band
MD_gamma_phase
MD_lfp_trace
MD_theta_band
MD_theta_phase
all_subjects
coherence_calculation_frequencies
cohort
first_timestamp
granger_calculation_frequencies
last_timestamp
lfp_timestamps
mPFC_gamma_band
mPFC_gamma_phase
mPFC_lfp_trace
mPFC_theta_band
mPFC_theta_phase
power_calculation_frequencies
recording
session_path
subject
tone_frames
tone_timestamps
vHPC_gamma_band
vHPC_gamma_phase
vHPC_lfp_trace
vHPC_theta_band
vHPC_theta_phase
video_ti

# Filtering out phase

In [82]:
lfp_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col and "timestamps" not in col and "calculation" not in col]

In [83]:
band_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "band" in col and "timestamps" not in col and "calculation" not in col]

In [84]:
phase_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "phase" in col and "timestamps" not in col and "calculation" not in col]

In [85]:
phase_columns = phase_columns + band_columns +lfp_columns

In [86]:
phase_columns

['BLA_gamma_phase',
 'BLA_theta_phase',
 'LH_gamma_phase',
 'LH_theta_phase',
 'MD_gamma_phase',
 'MD_theta_phase',
 'mPFC_gamma_phase',
 'mPFC_theta_phase',
 'vHPC_gamma_phase',
 'vHPC_theta_phase',
 'BLA_gamma_band',
 'BLA_theta_band',
 'LH_gamma_band',
 'LH_theta_band',
 'MD_gamma_band',
 'MD_theta_band',
 'mPFC_gamma_band',
 'mPFC_theta_band',
 'vHPC_gamma_band',
 'vHPC_theta_band',
 'BLA_lfp_trace',
 'LH_lfp_trace',
 'MD_lfp_trace',
 'mPFC_lfp_trace',
 'vHPC_lfp_trace']

In [87]:
for col in sorted(phase_columns):
    updated_item_col = "trial_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_and_post_lfp_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)


trial_and_post_BLA_gamma_band
trial_and_post_BLA_gamma_phase
trial_and_post_BLA_lfp_trace
trial_and_post_BLA_theta_band
trial_and_post_BLA_theta_phase
trial_and_post_LH_gamma_band
trial_and_post_LH_gamma_phase
trial_and_post_LH_lfp_trace
trial_and_post_LH_theta_band
trial_and_post_LH_theta_phase
trial_and_post_MD_gamma_band
trial_and_post_MD_gamma_phase
trial_and_post_MD_lfp_trace
trial_and_post_MD_theta_band
trial_and_post_MD_theta_phase
trial_and_post_mPFC_gamma_band
trial_and_post_mPFC_gamma_phase
trial_and_post_mPFC_lfp_trace
trial_and_post_mPFC_theta_band
trial_and_post_mPFC_theta_phase
trial_and_post_vHPC_gamma_band
trial_and_post_vHPC_gamma_phase
trial_and_post_vHPC_lfp_trace
trial_and_post_vHPC_theta_band
trial_and_post_vHPC_theta_phase


In [88]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_lfp_timestamps"]

0      [34280580, 34280600, 34280620, 34280640, 34280...
1      [34280580, 34280600, 34280620, 34280640, 34280...
2      [35280600, 35280620, 35280640, 35280660, 35280...
3      [35280600, 35280620, 35280640, 35280660, 35280...
4      [36380600, 36380620, 36380640, 36380660, 36380...
                             ...                        
243    [57964900, 57964920, 57964940, 57964960, 57964...
244    [60164920, 60164940, 60164960, 60164980, 60165...
245    [60164920, 60164940, 60164960, 60164980, 60165...
246    [61264940, 61264960, 61264980, 61265000, 61265...
247    [61264940, 61264960, 61264980, 61265000, 61265...
Name: trial_and_post_lfp_timestamps, Length: 248, dtype: object

In [89]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=phase_columns + ["lfp_timestamps"], errors="ignore")

In [90]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [91]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tone_start_frame
reward_start
reward_dispensed
tone_stop_frame
condition 
competition_closeness
get_reward_frame
out_reward_frame
notes
box_1_port_entry_frames
box_2_port_entry_frames
video_name
tone_start_timestamp
tone_stop_timestamp
box_1_port_entry_timestamps
box_2_port_entry_timestamps
current_subject
subject_tail_base
agent_tail_base
session_dir
experiment
sleap_name
video_id
agent
all_subjects
coherence_calculation_frequencies
cohort
first_timestamp
granger_calculation_frequencies
last_timestamp
power_calculation_frequencies
recording
session_path
subject
tone_frames
tone_timestamps
video_timestamps
baseline_start_timestamp
post_trial_end_timestamp
video_frame
trial_and_post_agent_locations
trial_and_post_agent_nose
trial_and_post_agent_nose_to_reward_port
trial_and_post_agent_thorax
trial_and_post_agent_thorax_velocity
trial_and_post_agent_to_reward_port_angle
trial_and_post_closebool_agent_nose_to_reward_port
trial_and_post_closebool_subject_nose_to_reward_port
trial_and_post_

In [94]:
TRIALS_AND_SPECTRAL_DF["trial_and_post_kmeans_cluster"].iloc[0].shape

(400,)

In [92]:
raise ValueError()

ValueError: 

# Filtering for rows that are in the video

In [ ]:
TRIALS_AND_SPECTRAL_DF["in_video"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["start_frame"] <= x["trial_video_frame"][0] <= x["stop_frame"], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["in_video"]].reset_index()

In [ ]:
TRIALS_AND_SPECTRAL_DF

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError()

# Filtering out spikes

In [ ]:
raise ValueError()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1], axis=1)
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1].T, axis=1)
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=["spike_clusters", "spike_times", "neuron_average_fr", "neuron_average_timestamps",], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"].iloc[0].shape